In [83]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets
import math

import numpy as np

In [106]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_samples=1000, n_features = 4, n_informative = 2, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=5)

In [107]:
classification_labels

array([0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,

In [108]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [109]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        

In [110]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [111]:
# Расчет энтропийного критерия

def entrop(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 0
    for label in classes:
        p = classes[label] / len(labels)
        impurity += p ** math.log(p, 2)
        
    impurity = -impurity
        
    return impurity

In [112]:
# Расчет качества

def quality(left_labels, right_labels, current, metric=gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current - p * metric(left_labels) - (1 - p) * metric(right_labels)

In [113]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [114]:
data = classification_data
n_features = data.shape[1]
for index in range(n_features):
    t_values = np.unique([row[index] for row in data])
t_values[1]

-3.287652375194488

In [115]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels, min_leaf, metric=gini):
    
    #  обозначим минимальное количество объектов в узле
    
    current = metric(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current, metric)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [116]:
# Построение дерева с помощью рекурсивной функции


def build_tree(data, labels, leaf_count, depth=1, max_depth=5, min_leaf=5, metric=gini, max_leaf_count=100):
    
    # Прекращение рекурсии если достигнута максимальное колличество листов
    if leaf_count > max_leaf_count:
        return Leaf(data, labels)
    
    #  Прекращение рекурсии если достигнуто ограничение на глубину дерева
    if depth > max_depth:
        leaf_count += 1
        return Leaf(data, labels)
        #return Node(index, t, true_branch, false_branch)
    
    quality, t, index = find_best_split(data, labels, min_leaf, metric)
    
    
        
    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        leaf_count += 1
        return Leaf(data, labels)
    
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t) 
    
    # Прекращение рекурсии если все объекты в листе относятся к одному классу
    if 1 not in true_labels or 1 not in false_labels:
        leaf_count += 1
        return Leaf(data, labels)
    
    quality, t, index = find_best_split(data, labels, min_leaf, metric)
    

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, depth+1, max_depth)
    false_branch = build_tree(false_data, false_labels, depth+1, max_depth)
    


    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [117]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [118]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [119]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [120]:
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [134]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [135]:
# Построим дерево по обучающей выборке используя критерий джинни
my_tree_gini = build_tree(train_data, train_labels, leaf_count=0, metric=gini)

In [136]:
# Получим ответы для обучающей выборки 
train_answers_gini = predict(train_data, my_tree_gini)

In [137]:
# И получим ответы для тестовой выборки
answers_gini = predict(test_data, my_tree_gini)

In [138]:
# Напечатаем ход дерева
print_tree(my_tree_gini)

Индекс 3
Порог 0.10665115923189461
--> True:
  Индекс 1
  Порог -1.3459121907528029
  --> True:
    Индекс 1
    Порог -1.675908596478551
    --> True:
      Прогноз: 0
    --> False:
      Прогноз: 0
  --> False:
    Прогноз: 0
--> False:
  Прогноз: 1


In [139]:
# Точность на обучающей выборке
train_accuracy = accuracy_metric(train_labels, train_answers_gini)
train_accuracy

88.14285714285714

In [140]:
# Точность на тестовой выборке
test_accuracy = accuracy_metric(test_labels, answers_gini)
test_accuracy

86.66666666666667

In [142]:
# Построим дерево по обучающей выборке используя метрику энтропиии
my_tree_entrop = build_tree(train_data, train_labels, leaf_count=0, metric=entrop)

In [143]:
train_answers_entrop = predict(train_data, my_tree_entrop)

In [144]:
answers_entrop = predict(test_data, my_tree_entrop)

In [145]:
print_tree(my_tree_entrop)

Индекс 3
Порог -1.3951377815806283
--> True:
  Прогноз: 0
--> False:
  Индекс 3
  Порог 0.10665115923189461
  --> True:
    Индекс 1
    Порог -1.3459121907528029
    --> True:
      Индекс 1
      Порог -1.675908596478551
      --> True:
        Прогноз: 0
      --> False:
        Прогноз: 1
    --> False:
      Прогноз: 0
  --> False:
    Прогноз: 1


In [146]:
train_accuracy = accuracy_metric(train_labels, train_answers_entrop)
train_accuracy

88.14285714285714

In [147]:
test_accuracy = accuracy_metric(test_labels, answers_entrop)
test_accuracy

88.66666666666667

Реализовал кретерии останова: Максимальное количество листьев, Максимальная глубина дерева, Минимальное количетсво обьектов в листе, Всё обьекты в листе одного класса.

Реализовал кретерий энетропии. Не обноружил разницы с использованием кретерия Джини. 